In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from scipy.ndimage import gaussian_filter1d
from pandas import DataFrame, Series
import matplotlib.pylab as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import datetime
import pyproj 
import random
import os
import io

%matplotlib inline
np.set_printoptions(precision=3 , suppress= True)

In [17]:
#initial parameters - path
folder_path = '/home/rafaela/internship/time_series/TimeSeries-insar/egm_files'
file_path = os.path.join(folder_path, 'EGMS_L3_E44N25_100km_U.csv')
teste = pd.read_csv(file_path)

teste_reduced = teste.iloc[:,list(range(1, 4)) + list(range(11, 375))]
print(teste_reduced)

        easting  northing  height  20160105  20160111  20160117  20160123  \
0       4499150   2596450  2186.4       3.2       5.2       3.7       1.6   
1       4499350   2596550  2060.1       4.3       2.8       2.5       2.9   
2       4499150   2596550  2112.4      -2.2      -2.0      -1.5      -2.1   
3       4498950   2596550  2075.7      -1.2      -1.7      -1.4      -0.9   
4       4498850   2596550  2024.0       1.4       1.7      -1.9      -2.3   
...         ...       ...     ...       ...       ...       ...       ...   
209839  4415850   2516650   565.8      -1.0      -0.2       0.0       0.8   
209840  4415950   2516650   563.0       1.0       0.5       0.5       0.9   
209841  4416050   2516650   551.3       0.8       1.4       1.2      -0.2   
209842  4416150   2516650   537.0      -1.5      -3.2      -3.1       1.0   
209843  4416250   2516750   535.2      -3.2      -1.8      -1.2       0.1   

        20160129  20160204  20160210  ...  20211029  20211104  20211110  \


In [14]:
#define the ETRS89-LAEA and the UTM projection
etrs89_laea = pyproj.CRS("EPSG:3035")
wgs84 = pyproj.CRS("EPSG:4326")
utm_32 = pyproj.CRS("EPSG:23032")

In [15]:
teste_new = teste_reduced.copy()

transformer = pyproj.Transformer.from_crs(etrs89_laea, wgs84)
teste_new["latitude"], teste_new["longitude"] = transformer.transform(teste_new["northing"].to_numpy(), teste_new["easting"].to_numpy())

print(teste_new)

        easting  northing  height  20160105  20160111  20160117  20160123  \
0       4499150   2596450  2186.4       3.2       5.8       3.2       1.6   
1       4499350   2596550  2060.1       8.4       6.5       6.2       4.6   
2       4499150   2596550  2112.4      -1.0      -0.8      -2.5      -3.2   
3       4498950   2596550  2075.7       1.7       0.9       0.3      -1.1   
4       4498850   2596550  2024.0       1.1       1.5       2.7       0.8   
...         ...       ...     ...       ...       ...       ...       ...   
209839  4415850   2516650   565.8      -0.9      -0.2      -0.1      -0.8   
209840  4415950   2516650   563.0      -0.0      -0.4      -0.4      -0.4   
209841  4416050   2516650   551.3      -1.2      -0.6      -0.4       2.4   
209842  4416150   2516650   537.0       2.0      -0.3      -1.1      -2.4   
209843  4416250   2516750   535.2      -1.0       0.2       0.5      -1.1   

        20160129  20160204  20160210  ...  20211110  20211116  20211122  \


In [16]:
latitude_min = 45.81
latitude_max = 45.84
longitude_min = 12.10
longitude_max = 12.16

teste_area_selected = teste_new[((teste_new['latitude'] >= latitude_min) & (teste_new['latitude'] <= latitude_max) &
                  (teste_new['longitude'] >= longitude_min) & (teste_new['longitude'] <= longitude_max))]

print(teste_area_selected)

       easting  northing  height  20160105  20160111  20160117  20160123  \
31809  4486250   2527350   131.8      -1.2      -0.0       1.1       1.7   
31810  4486550   2527350   157.3      -3.8      -4.5      -3.5      -2.1   
31811  4486150   2527350   132.5      -0.3       0.0      -0.1      -0.0   
31812  4486950   2527450   150.4       1.9       1.6       1.7       0.8   
31813  4486850   2527450   177.8       6.9       8.0       2.1       1.1   
...        ...       ...     ...       ...       ...       ...       ...   
34394  4485150   2524850   324.3       0.3       2.0       0.7      -0.7   
34395  4485250   2524950   315.5      -0.7      -1.4       0.3       2.5   
34396  4484850   2524950   327.6       1.3       1.0       0.6       0.8   
34397  4485150   2524950   323.0       3.3       2.8       3.3       4.6   
34398  4485050   2524950   326.4      -1.0      -0.6      -0.2       0.7   

       20160129  20160204  20160210  ...  20211110  20211116  20211122  \
31809       0

In [8]:
def selec_points(teste_area_selected, indice_number):
    points_location = teste_area_selected.iloc[indice_number, 367:369]
    teste_point = teste_area_selected.iloc[indice_number, 3:367]
    teste_point.index = pd.to_datetime(teste_point.index, format='%Y%m%d')
    points_dataframe = pd.DataFrame(teste_point, columns=[teste_point.name])
    
    return (points_dataframe, points_location)
    
num_points = 332
indicies = random.choices(range(len(teste_area_selected)), k = num_points)
points_dictionaire = {}
points_location_list = []

for indice_number in indicies:
    points_dataframe, points_location = selec_points(teste_area_selected, indice_number)
    points_dictionaire[indice_number] = points_dataframe
    points_location_list.append(points_location)

In [10]:
print(len(points_dictionaire))

221


In [ ]:
points_location_df = pd.DataFrame(points_location_list)
points_file_path = "/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/points_location_list_east.csv"
points_location_df.to_csv(points_file_path, index=True)

In [ ]:
def linear_detrend(column_name, indice_data):
 
    #getting the linear model to substract from the original time series:    
    x = np.arange(len(indice_data)).reshape(-1, 1)
    y = indice_data.values.reshape(-1, 1)
    
    linear_model = LinearRegression().fit(x, y)
    regression_line = linear_model.predict(x)
    
    detrended_values = y.flatten() - regression_line.flatten()
    #detrended_values = station_data[f'{station_name}_east(m)'].values - regression_line.flatten()
    
    detrended_dataframe = pd.DataFrame(
        detrended_values, 
        index=indice_data.index, 
        columns=[f'{column_name}_east(mm)_detrended'])
        
    return detrended_dataframe


#dictionaries to store the data
detrended_data_dict = {}

#calling the function
for indice_number, indice_data in points_dictionaire.items():
    
    column_name = indice_data.columns[0]
    detrended_dataframe = linear_detrend(
        column_name, 
        indice_data)
    detrended_data_dict[column_name] = detrended_dataframe

#printing dataframe
#stations detrended
for indice_number, indice_data in detrended_data_dict.items():
    print(f'{indice_number}: \n{indice_data}\n')

In [ ]:
def remove_outliers(indice_number, indice_data):
    limiar_z = 4.0
    
    
    selecting_outliers = indice_data[(np.abs(indice_data-indice_data.mean()) 
                                              < limiar_z*indice_data.std())]
    
    
    outliers_removed_dataframe = pd.DataFrame(selecting_outliers, 
                                              index=indice_data.index)
    
    outliers_removed_dataframe = outliers_removed_dataframe.rename(
        columns={f'{indice_number}_east(mm)_detrended':f'{indice_number}_east(mm)'})  
    
    outliers_removed_dataframe.dropna(inplace=True)
    
    return outliers_removed_dataframe

#dictionaries to store the data
area_outliers_removed_dict = {}

#calling the function
for indice_number, indice_data in detrended_data_dict.items():
    outliers_removed_dataframe = remove_outliers(
        indice_number, 
        indice_data)
    area_outliers_removed_dict[indice_number] = outliers_removed_dataframe

#printing dataframe
#stations detrended
#for indice_number, indice_data in area_outliers_removed_dict.items():
    #print(f'{indice_number}: \n{indice_data}\n')

In [ ]:
def get_rolling_mean(indice_number, indice_data):
    
    area_rolling_mean = indice_data.rolling(2, min_periods=1).mean()
    
    areas_rolling_mean_dataframe = pd.DataFrame(area_rolling_mean, 
                                              index=indice_data.index)
    
    areas_rolling_mean_dataframe = areas_rolling_mean_dataframe.rename(
        columns={f'{indice_number}_east(mm)':f'{indice_number}_east(mm)_rm'})
    
    areas_rolling_mean_dataframe.dropna(inplace=True)
    
    return areas_rolling_mean_dataframe

#dictionaries to store the data
areas_rolling_mean_dict = {}

#calling the function
for indice_number, indice_data in area_outliers_removed_dict.items():
    areas_rolling_mean_dataframe = get_rolling_mean(indice_number, indice_data)
    areas_rolling_mean_dict[indice_number] = areas_rolling_mean_dataframe

#printing dataframe
#stations detrended
for indice_number, indice_data in areas_rolling_mean_dict.items():
    print(f'{indice_number}: \n{indice_data}\n')

In [ ]:
#getting the average - all points 
all_rm_data = pd.DataFrame()

for indice_number, indice_data in areas_rolling_mean_dict.items():
    all_rm_data[indice_number] = indice_data.squeeze()

avg_rm = all_rm_data.mean(axis=1).rolling(2, min_periods=1).mean()
avg_rm_df = pd.DataFrame(avg_rm)
avg_rm_df.columns = ['avg_east(mm)']

print(avg_rm_df)

In [ ]:
#removing the avg
points_avgrm_reduced = pd.DataFrame()

for indice_number, indice_data in areas_rolling_mean_dict.items():
    column_name = f"{indice_number}_east(mm)_rm"
    
    if column_name in indice_data:
        try:
            points_avgrm_reduced[indice_number] = indice_data[column_name] - avg_rm
        except Exception as e:
            print(f"Erro ao processar {indice_number}: {e}")
    else:
        print(f"Warning: {column_name} not found in indice_data")

print(points_avgrm_reduced)

In [ ]:
#checking the points plots - AFTER the reduction

colors = sns.color_palette("bright", len(points_avgrm_reduced))
folder = '/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/insar_ts_plots/east/points_after_avg_rolling_mean'

for i, (column, color) in enumerate(zip(points_avgrm_reduced.columns, colors)):
    plt.figure(figsize=(25, 5))
    plt.plot(points_avgrm_reduced.index, points_avgrm_reduced[column], 'o', color=color, lw=2, label=column) 
    
    plt.title(f'{column}', fontsize = 20)
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, label='Reference')
    plt.xlabel('Time', fontsize = 15)
    plt.xticks(fontsize=15)
    plt.ylabel('East(mm)', fontsize = 15)                                       
    plt.yticks(fontsize=15)
    #plt.ylim(-0.008,0.008)
    #plt.xlim(pd.Timestamp('2011-01-15'), pd.Timestamp('2018-07-15'))

    plt.legend()
    
    fl = os.path.join(folder, f'{column}.svg')
    f = io.BytesIO()
    #plt.savefig(fl, format="svg")
    
    plt.show()

In [ ]:
def create_station_file (
    points_avgrm_reduced,
    index_selected, 
    station_name,
    folder_path):
    
    station_file = points_avgrm_reduced[index_selected].squeeze()
    station_file_df = pd.DataFrame(station_file)
    station_file_df.columns = [index_selected]
    
    return station_file_df

station_file_dict = {}
indexes_selected = []

if not indexes_selected:
    indexes_selected = list(points_avgrm_reduced.keys())

file_station_name = 'canv'

for index_selected, station_name, in zip(indexes_selected, [file_station_name]*len(indexes_selected)):
    
    station_file_df = create_station_file(points_avgrm_reduced,
                                          index_selected, 
                                          station_name,
                                          folder_path)
    
    station_file_dict[index_selected] = station_file_df

In [ ]:
station_name = 'sper'
folder_path = f'/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/'


concatenated_df = pd.concat(station_file_dict.values(), axis=1)
file_path = os.path.join(folder_path, f'{station_name}_east_4000.csv')
#concatenated_df.to_csv(file_path, index=True)

In [ ]:
#Saving the graphics of the points selected

colors = sns.color_palette("bright", len(mgrd))
folder = '/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/insar_ts_plots/east/points_after_avg_rolling_mean/sper/graphics/'

for i, (column, color) in enumerate(zip(mgrd.columns, colors)):
    plt.figure(figsize=(25, 5))
    plt.plot(mgrd.index, mgrd[column], 'o', color=color, lw=2, label=column) 
    
    plt.title(f'{column}', fontsize = 20)
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, label='Reference')
    plt.xlabel('Time', fontsize = 15)
    plt.xticks(fontsize=15)
    plt.ylabel('East(mm)', fontsize = 15)                                       
    plt.yticks(fontsize=15)
    #plt.ylim(-0.008,0.008)

    plt.legend()
    
    fl = os.path.join(folder, f'{column}.svg')
    f = io.BytesIO()
    #plt.savefig(fl, format="svg")
    
    plt.show()

In [ ]:
avg_points_selec = pd.DataFrame()

for indice_number, indice_data in mgrd.items():
    avg_points_selec[indice_number] = indice_data.squeeze()

avg_station = avg_points_selec.mean(axis=1).rolling(2, min_periods=1).mean()
avg_station_df = pd.DataFrame(avg_station)
avg_station_df.columns = ['avg2_east(mm)']

print(avg_station_df)

In [ ]:
folder = '/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/insar_ts_plots/east/points_after_avg_rolling_mean/sper/'

plt.figure(figsize=(25, 5))

plt.plot(avg_rm.index, avg_rm, color=color, lw=2) 
plt.title('AVG 1 - MGRD 4000p', fontsize = 20)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1, label='Reference')
plt.xlabel('Time', fontsize = 15)
plt.xticks(fontsize=15)
plt.ylabel('East(mm)', fontsize = 15)                                       
plt.yticks(fontsize=15)
#plt.ylim(-0.008,0.008)

plt.legend()
    
fl = os.path.join(folder, 'AVG - MGRD-4000.svg')
f = io.BytesIO()
#plt.savefig(fl, format="svg")
    
plt.show()

In [ ]:
folder = '/home/rafaela/internship/time_series/TimeSeries-insar/egm_files/insar_ts_plots/east/points_after_avg_rolling_mean/sper/'

plt.figure(figsize=(25, 5))

plt.plot(avg_station_df.index, avg_station_df, 'o', color=color, lw=2) 
plt.title('AVG 2 - MGRD 4000p', fontsize = 20)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1, label='Reference')
plt.xlabel('Time', fontsize = 15)
plt.xticks(fontsize=15)
plt.ylabel('East(mm)', fontsize = 15)                                       
plt.yticks(fontsize=15)
#plt.ylim(-0.008,0.008)

plt.legend()
    
fl = os.path.join(folder, 'AVG - MGRD-4000.svg')
f = io.BytesIO()
#plt.savefig(fl, format="svg")
    
plt.show()

In [ ]:
#removing the avg
mgrd_avgrm_reduced = pd.DataFrame()
mgrd_avgrm_reduced = mgrd - avg_station_df

print(mgrd_avgrm_reduced)